In [37]:
import pandas as pd
import sys
import csv
from statistics import median

In [38]:
batchsizes = [1,2,4,8,16]

In [39]:
data = pd.read_csv('gfm_all_ssr_results.csv')

In [40]:
print(set(data['project'].tolist()))

{'loomio', 'puppet', 'orbeon-forms', 'rubinius', 'metasploit-framework', 'sufia', 'jruby', 'vagrant', 'rails', 'concerto', 'cloud_controller_ng', 'cloudify', 'graylog2-server', 'fog', 'opal', 'geoserver', 'gradle', 'sonarqube', 'open-build-service'}


In [41]:
data['project_reqd_builds'] = pd.to_numeric(data['project_reqd_builds'], downcast="float", errors='coerce')
data["project_missed_builds"] = pd.to_numeric(data["project_missed_builds"], downcast="float", errors='coerce')
data["project_saved_builds"] = pd.to_numeric(data["project_saved_builds"], downcast="float", errors='coerce')
data['batch_delays'] = pd.to_numeric(data["batch_delays"], downcast="float", errors='coerce')

In [42]:
delay_avg = []
delays = data['project_delays'].tolist()

for i in delays:
    if i == '[]':
        delay_avg.append(0)
    else:
        row = i[1:-1].split(', ')
        row = [int(x) for x in row]
        delay_avg.append(sum(row))
data['total_delay'] = delay_avg

In [43]:
# delay_avg = []
# delays = data['batch_delays'].tolist()

# for i in delays:
#     if i == '[]':
#         delay_avg.append(0)
#     else:
#         row = [x for x in list(i)[1:-1] if (x != ',' and x != ' ')]
#         row = [int(x) for x in row]
#         delay_avg.append(sum(row))
data['total_batch_delay'] = data['batch_delays'].tolist()

In [44]:
project_list = set(data['project'].tolist())
batching_alg = set(data['algorithm'].tolist())

In [45]:
data

,project,algorithm,batch_size,confidence,project_reqd_builds,project_missed_builds,project_build_duration,project_saved_builds,project_delays,testall_size,batch_delays,total_delay,total_batch_delay
0,vagrant,BATCHBISECT,1,2,50.352112,1.760563,20360,49.647888,"[2, 2, 2, 2, 2]",284,0.0,10,0.0
1,vagrant,BATCHBISECT,1,3,34.154930,1.760563,13570,65.845070,"[3, 2, 2, 2, 3]",284,0.0,12,0.0
2,vagrant,BATCHBISECT,1,4,25.352112,2.464789,10476,74.647888,"[4, 3, 2, 3, 2, 2, 3]",284,0.0,19,0.0
3,vagrant,BATCHBISECT,1,5,20.070423,2.816901,8287,79.929581,"[2, 3, 5, 4, 5, 4, 3, 4]",284,0.0,30,0.0
4,vagrant,BATCHBISECT,1,6,16.901409,2.464789,6773,83.098595,"[6, 5, 4, 2, 4, 6, 6]",284,0.0,33,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3244,geoserver,BATCHSTOP4,16,16,110.119049,1.190476,1333599,4.166667,"[5, 7]",168,1201.0,12,1201.0
3245,geoserver,BATCHSTOP4,16,17,110.119049,1.190476,1333599,4.166667,"[5, 7]",168,1201.0,12,1201.0
3246,geoserver,BATCHSTOP4,16,18,110.119049,1.190476,1333599,4.166667,"[5, 7]",168,1201.0,12,1201.0
3247,geoserver,BATCHSTOP4,16,19,110.119049,1.190476,1333599,4.166667,"[5, 7]",168,1201.0,12,1201.0


In [46]:
result_file = open('final_gfm_ssr_results.csv', 'w')
res_headers = ['project', 'method', 'algorithm', 'batch_size', 'builds_reqd', 'batch_delays', 'sbs_delays', 'confidence']
res_writer = csv.writer(result_file)
res_writer.writerow(res_headers)

84

In [47]:
print(len(project_list))

19


In [48]:
for p in project_list:
    p_name = p + '.csv'
    alg_scores = {}

    p_data = data[data['project'] == p]
    #print(p_data)
    for alg in batching_alg:

        alg_data = p_data[p_data['algorithm'] == alg]
        #print(alg_data)

        for b in batchsizes:

            if alg == 'BATCH4':
                if b != 4:
                    continue

            if alg == 'BATCHSTOP4':
                if b < 4:
                    continue

            batch_data = alg_data[ alg_data['batch_size'] == b]

            reqd_list = batch_data['project_reqd_builds'].tolist()
            delay_list = batch_data['total_delay'].tolist()
            batch_delay_list = batch_data['total_batch_delay'].tolist()
            min_distance = sys.maxsize

            best = [p_name, 'ssr', alg, b, 0, 0, 0, 0]
            best_i = 0

            for i in range(len(reqd_list)):
                distance = ((reqd_list[i]**2)+((delay_list[i]+batch_delay_list[i])**2))**0.5
                if distance < min_distance:
                    best[4] = reqd_list[i]
                    best[5] = batch_delay_list[i]
                    best[6] = delay_list[i]
                    best[7] = i+2
                    best_i = i
                    min_distance = distance

            res_writer.writerow(best)
            print(best)

['loomio.csv', 'ssr', 'BATCHSTOP4', 4, 37.56097412109375, 150.0, 55, 10]
['loomio.csv', 'ssr', 'BATCHSTOP4', 8, 33.65853500366211, 364.0, 115, 19]
['loomio.csv', 'ssr', 'BATCHSTOP4', 16, 58.53658676147461, 1171.0, 47, 20]
['loomio.csv', 'ssr', 'BATCH4', 4, 37.56097412109375, 150.0, 55, 10]
['loomio.csv', 'ssr', 'BATCHBISECT', 1, 38.53658676147461, 0.0, 33, 3]
['loomio.csv', 'ssr', 'BATCHBISECT', 2, 32.19512176513672, 40.0, 44, 6]
['loomio.csv', 'ssr', 'BATCHBISECT', 4, 40.4878044128418, 150.0, 55, 10]
['loomio.csv', 'ssr', 'BATCHBISECT', 8, 36.585365295410156, 364.0, 115, 19]
['loomio.csv', 'ssr', 'BATCHBISECT', 16, 63.414634704589844, 1171.0, 47, 20]
['puppet.csv', 'ssr', 'BATCHSTOP4', 4, 4.34782600402832, 18.0, 6, 16]
['puppet.csv', 'ssr', 'BATCHSTOP4', 8, 4.34782600402832, 57.0, 22, 18]
['puppet.csv', 'ssr', 'BATCHSTOP4', 16, 14.492753982543945, 240.0, 0, 19]
['puppet.csv', 'ssr', 'BATCH4', 4, 4.34782600402832, 18.0, 6, 16]
['puppet.csv', 'ssr', 'BATCHBISECT', 1, 5.7971014976501465,

In [49]:
data

,project,algorithm,batch_size,confidence,project_reqd_builds,project_missed_builds,project_build_duration,project_saved_builds,project_delays,testall_size,batch_delays,total_delay,total_batch_delay
0,vagrant,BATCHBISECT,1,2,50.352112,1.760563,20360,49.647888,"[2, 2, 2, 2, 2]",284,0.0,10,0.0
1,vagrant,BATCHBISECT,1,3,34.154930,1.760563,13570,65.845070,"[3, 2, 2, 2, 3]",284,0.0,12,0.0
2,vagrant,BATCHBISECT,1,4,25.352112,2.464789,10476,74.647888,"[4, 3, 2, 3, 2, 2, 3]",284,0.0,19,0.0
3,vagrant,BATCHBISECT,1,5,20.070423,2.816901,8287,79.929581,"[2, 3, 5, 4, 5, 4, 3, 4]",284,0.0,30,0.0
4,vagrant,BATCHBISECT,1,6,16.901409,2.464789,6773,83.098595,"[6, 5, 4, 2, 4, 6, 6]",284,0.0,33,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3244,geoserver,BATCHSTOP4,16,16,110.119049,1.190476,1333599,4.166667,"[5, 7]",168,1201.0,12,1201.0
3245,geoserver,BATCHSTOP4,16,17,110.119049,1.190476,1333599,4.166667,"[5, 7]",168,1201.0,12,1201.0
3246,geoserver,BATCHSTOP4,16,18,110.119049,1.190476,1333599,4.166667,"[5, 7]",168,1201.0,12,1201.0
3247,geoserver,BATCHSTOP4,16,19,110.119049,1.190476,1333599,4.166667,"[5, 7]",168,1201.0,12,1201.0


In [50]:
result_file = open('ssr_delay_lists.csv', 'w')
res_headers = ['project', 'method', 'algorithm', 'batch_size', 'sbs_delays', 'median_delays', 'confidence']
res_writer = csv.writer(result_file)
res_writer.writerow(res_headers)

73

In [58]:
lines = []

In [63]:
for p in project_list:
    p_name = p + '.csv'
    alg_scores = {}

    p_data = data[data['project'] == p]
    #print(p_data)
    for alg in batching_alg:

        alg_data = p_data[p_data['algorithm'] == alg]
        #print(alg_data)

        for b in batchsizes:

            if alg == 'BATCH4':
                if b != 4:
                    continue

            if alg == 'BATCHSTOP4':
                if b < 4:
                    continue

            batch_data = alg_data[ alg_data['batch_size'] == b]

            reqd_list = batch_data['project_reqd_builds'].tolist()
            delay_list = batch_data['total_delay'].tolist()
            batch_delay_list = batch_data['total_batch_delay'].tolist()
            ssr_delays = batch_data['project_delays'].tolist()
            min_distance = sys.maxsize

            best = [p_name, 'ssr', alg, b, 0, [], 0]
            best_i = 0

            for i in range(len(reqd_list)):
                distance = ((reqd_list[i]**2)+((delay_list[i]+batch_delay_list[i])**2))**0.5
                if distance < min_distance:
                    best[4] = ssr_delays[i]
                    print(median(ssr_delays[i]))
                    #best[5] = median(ssr_delays[i])
                    best[6] = i+2
                    best_i = i
                    min_distance = distance

            lines.append(best)
            if alg == 'BATCH4':
                print(best)

<class 'str'>


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [57]:
ssr_delay = pd.DataFrame(lines, columns=['project', 'method', 'algorithm', 'batch_size', 'sbs_delays', 'confidence'])

In [34]:
ssr_delay.to_csv('trial.csv')